In [1]:
#quickstart
from pathlib import Path
import pandas as pd
from bids import BIDSLayout
from joblib import Parallel, delayed
import nilearn
import nibabel as nib
import nltools.prefs as prf
from nilearn import datasets, image, plotting, input_data
from nilearn.masking import apply_mask
from nilearn.image import resample_to_img
from nilearn.image import concat_imgs, index_img
import numpy as np
import matplotlib.pylab as plt

import rpy2
from rpy2.robjects.packages import importr
from pymer4 import Lmer
psych = importr('psych')


import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

import os
import glob
import seaborn as sns
from nltools.data import Brain_Data, Design_Matrix, Adjacency
from nltools.mask import expand_mask, roi_to_brain
from nltools.stats import zscore
from scipy.stats import binom, ttest_1samp
from sklearn.metrics import pairwise_distances
from nltools.stats import fdr, one_sample_permutation
from copy import deepcopy
from nltools.utils import get_anatomical
import nltools.plotting as nlplt
from datetime import date
from joblib import Parallel, delayed

from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

from nilearn.input_data import NiftiMapsMasker,NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

import sys
import pandas as pd
import rpy2
from rpy2.robjects.packages import importr
from pymer4 import Lmer
psych = importr('psych')
import numpy as np

import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

import os
import glob
import seaborn as sns
from statistics import mean
import matplotlib.style as style

#dat_paths = pd.read_csv('/data/MBDU/midla/notebooks/campcc/dat_paths_11272019.csv') 
#atlas = image.load_img('/data/MBDU/midla/notebooks/campcc/hox_msdl_atlas.nii.gz')
#sub1 = image.load_img('/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm/sub-20900/out/fmrip

%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))


/gpfs/gsfs11/users/MBDU/midla/env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
measure_df = pd.read_csv('/data/MBDU/midla/notebooks/campcc/test_retest/resting_state/icc_by_scan_pair_cc.csv')

In [8]:
pair = measure_df['pair'][0]

In [10]:
dat_paths = pd.read_csv('/data/MBDU/midla/data/derivatives/midla/primary_20200325/for_chris_svpair.csv')

In [20]:
subcounts = pd.DataFrame(dat_paths.drop_duplicates(['subject']).sv_pair.value_counts())

In [29]:
len(measure_df.loc[measure_df.pair == pair].loc[measure_df.ICC > .5])

24475

In [56]:
five = lambda x: 100*(len(x.loc[x > .5]/len(x))/len(x))
five.__name__ = "% Connections > .5"
seven = lambda x: 100*(len(x.loc[x > .7])/len(x))
seven.__name__ = "% Connections > .7"

In [57]:
table = measure_df.loc[measure_df.type == 'ICC2'].groupby('pair')[['ICC']].agg(['mean','std','count',five,seven])

In [58]:
table.merge(subcounts, left_on = 'pair', right_index = True)

,"(ICC, mean)","(ICC, std)","(ICC, count)","(ICC, % Connections > .5)","(ICC, % Connections > .7)",sv_pair
pair,,,,,,
3TA__DV25.1_R01_1617.b__3TA__DV25.1_R01_1617.b,0.413495,0.196380,6048,36.607143,5.406746,14
3TA__DV25.1_R01_1617.b__3TA__DV26.0_R01_1725.a,0.392802,0.169218,4032,26.984127,2.579365,7
3TA__DV25.1_R01_1617.b__3TA__DV26.0_R03_1831.b,0.254753,0.322051,20160,25.595238,15.029762,2
3TA__DV26.0_R01_1725.a__3TA__DV26.0_R01_1725.a,0.511628,0.137169,14112,57.242063,6.250000,17
3TA__DV26.0_R01_1725.a__3TA__DV26.0_R03_1831.b,0.527889,0.130930,12096,62.648810,6.101190,15
3TA__DV26.0_R03_1831.b__3TA__DV26.0_R03_1831.b,0.486918,0.140026,16128,49.355159,4.861111,12
3TC__DV22.0_V02_1122.a__3TC__DV26.0_R01_1725.a,0.389586,0.262159,2016,36.954365,13.392857,6
3TC__DV22.0_V02_1122.a__3TC__DV26.0_R03_1831.b,0.457498,0.369408,8064,50.545635,36.458333,2
3TC__DV26.0_R01_1725.a__3TC__DV26.0_R01_1725.a,0.481180,0.099083,10080,43.700397,0.644841,24
